# MuJoCo

In [68]:
import mujoco_py
import os
import numpy as np

Get MuJoCo `MjSim` from either MuJoCo or RoboSuite

In [70]:
xml_path = os.path.join('models', 'cloth.xml')
xml_path

'models/cloth.xml'

In [71]:
model = mujoco_py.load_model_from_path(xml_path)
sim = mujoco_py.MjSim(model)

# Collecting Trajectories

TODO: Look into `MjSimPool`

In [131]:
_joint_names = ['joint_1', 'joint_2', 'joint_3', 'joint_4', 'joint_5', 'joint_6', 'joint_7']
_cloth_keypoints = ['S0_0', 'S8_0', 'S0_8', 'S8_8']

"""
Collect state variables at the current step of the simulation.
    Returns: A vector of joint qpos, qvel, and keypoint positions.
"""
def get_features(sim):
    joint_qpos = np.array([sim.data.get_joint_qpos(joint) for joint in _joint_names])
    joint_qvel = np.array([sim.data.get_joint_qvel(joint) for joint in _joint_names])
    keypoint_xpos = np.array([sim.data.get_site_xpos(site) for site in _cloth_keypoints]).flatten()
    return np.concatenate([joint_qpos, joint_qvel, keypoint_xpos])

"""
Collect a trajectory of length `n`, apply control determined by `us` at each time step.
    sim: MuJoCo MjSim.
    n: Scalar length of the trajectory.
    us: (n x NJOINTS) array of inputs.
"""
def get_trajectory(sim, n, us):
    sim.reset()
    traj = []
    for i in range(n):
        sim.data.ctrl[:] = us[i]
        traj.append(get_features(sim))
        sim.step()
    return np.array(traj)

In [132]:
# Collect a trajectory of length 100 with no inputs.
n = 100
njoints = sim.data.ctrl.shape[0]
us = np.zeros((n, njoints))
traj = get_trajectory(sim, n, us)

# RoboSuite

In [40]:
from robosuite.models import MujocoWorldBase

world = MujocoWorldBase()

from robosuite.models.robots import IIWA

mujoco_robot = IIWA()
mujoco_robot.set_base_xpos([0, 0, 0])
world.merge(mujoco_robot)

In [41]:
model = world.get_model(mode="mujoco_py")

In [133]:
from mujoco_py import MjSim, MjViewer

sim = MjSim(model)